sales = unit of sales

revenue = amount of money

Original plan:

revenue at each store ~ everything we can get

From the original dataset, we will:

1. manipulate the dataframe to the way we desire.

2. combine daily usales to weekly sales, pair each week with events (joined from calendar.csv).

3. pair each weekly sales with its correspondance price (joined from sales_price.csv).

4. sales * price = weekly revenue for the item.

5. add all revenues from the same store for the total revenue of that store.

for each data point, we will have: revenue, store_id, week_id, and event .

In this case, dependent var will be the revenue, independent var will be the rest of them. We will have store_id and event as categorical and create dummy var for them (9 + 4 cols). Note that we have to keep all four columns for event because there is a NaN type. We will treat week_id as numerical because there are 278 different values. 

The size of training data will be 278 weeks * 10 stores = 2780 rows and 9+4+1+1=15 columns (2780, 15).

The size of testing data will be 4 weeks * 10 stores = 40 rows.

My concern is:

1. weekly_id has cyclic effect as sales at the same period per year will be similar. For example, sales will always boost during holiday seasons and drop down a bit after that. 

2. it is not true that as time goes on, the revenue will have a upward or downward trend over the period of time of the dataset.

Intuitively, our numerical input is not linear to the output.


My plan proposed during the meeting is:

sale (in unit) ~ everything else

everything else = price + department_id + store_id + weekly_id + event

From the original dataset:

1. manipulate the dataframe to the way we desire.

2. combine daily usales to weekly sales, pair each week with events (joined from calendar.csv)

3. pair each weekly sales with its correspondance price (joined from sales_price.csv)

And that's it. It is actualy easier than the first plan because for the first plan, to get the revenue, we have to visit price no matter how.

for each data point, we will have: sales, store_id, department_id, week_id, and event.

In this case, dependent var will be the sales, independent var will be the rest of them. We will have store_id, department_id and event as categorical and create dummy var for them (9 + 6 + 4 cols). We will treat week_id as numerical or categorical (not sure yet).

The size of training data will be 278 weeks * 3049 items * 10 stores = 8476220 rows and 9+6+4+1+1=21 columns (8476220, 21).

The size of testing data will be 4 weeks * 3049 items * 10 stores = 121960 rows.

The benefit of doing this is:

1. price is has linear relationship with sales.

2. data preprocessing load is no higher than the first plan.

3. more rows could lead to more accurate model (maybe).


In [2]:
278*3049*10

8476220

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('../Team-1/Retail Store Sales Forecast and Accuracy Analysis/Data/sales_train_evaluation.csv')

In [5]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [6]:
df_day = df.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
        var_name='day', 
        value_name='sales')

In [7]:
df_day.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [8]:
df_day_copy = df_day.copy()
# df_day = df_day_copy.copy()

In [9]:
df_day.shape

(59181090, 8)

In [10]:
30490*1941

59181090

In [11]:
df_day.drop(['id','cat_id','state_id'], axis=1, inplace=True)

In [12]:
df_day.head()

,item_id,dept_id,store_id,day,sales
0,HOBBIES_1_001,HOBBIES_1,CA_1,d_1,0
1,HOBBIES_1_002,HOBBIES_1,CA_1,d_1,0
2,HOBBIES_1_003,HOBBIES_1,CA_1,d_1,0
3,HOBBIES_1_004,HOBBIES_1,CA_1,d_1,0
4,HOBBIES_1_005,HOBBIES_1,CA_1,d_1,0


In [13]:
df_day['day'] = df_day['day'].apply(lambda x: int(x.split('_')[1]))

In [14]:
df_day['week'] = df_day['day']//7+1

In [15]:
df_day['day']

0              1
1              1
2              1
3              1
4              1
            ... 
59181085    1941
59181086    1941
59181087    1941
59181088    1941
59181089    1941
Name: day, Length: 59181090, dtype: int64

In [16]:
df_day['week']

0             1
1             1
2             1
3             1
4             1
           ... 
59181085    278
59181086    278
59181087    278
59181088    278
59181089    278
Name: week, Length: 59181090, dtype: int64

In [17]:
df_day.head()

,item_id,dept_id,store_id,day,sales,week
0,HOBBIES_1_001,HOBBIES_1,CA_1,1,0,1
1,HOBBIES_1_002,HOBBIES_1,CA_1,1,0,1
2,HOBBIES_1_003,HOBBIES_1,CA_1,1,0,1
3,HOBBIES_1_004,HOBBIES_1,CA_1,1,0,1
4,HOBBIES_1_005,HOBBIES_1,CA_1,1,0,1


From here we can join df_cal with df_day by column 'd' to get the wm_yr_wk and event. After that, we can do something like below to get the weekly sales. With 'wm_yr_wk', 'item_id', 'store_id' we can get price from sell_prices.csv.

after that item_id will be replaced by price in df_week so we have our num var.

Things to pay attention: 

1. find a way to aggregate event weekly.

2. the numerical week is the one I mentioned to be treated as input. We need somehow find a way to link it with wm_yr_week to make sure it is accurate. Right now I think it needs calibration.

In [18]:
df_cal = pd.read_csv('../Team-1/Retail Store Sales Forecast and Accuracy Analysis/Data/calendar.csv')

In [19]:
df_cal['d'] = df_cal['d'].apply(lambda x: int(x.split('_')[1]))

In [20]:
df_cal.shape

(1969, 14)

In [21]:
df_cal.head(20)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,5,NaN,NaN,NaN,NaN,1,0,1
5,2011-02-03,11101,Thursday,6,2,2011,6,NaN,NaN,NaN,NaN,1,1,1
6,2011-02-04,11101,Friday,7,2,2011,7,NaN,NaN,NaN,NaN,1,0,0
7,2011-02-05,11102,Saturday,1,2,2011,8,NaN,NaN,NaN,NaN,1,1,1
8,2011-02-06,11102,Sunday,2,2,2011,9,SuperBowl,Sporting,NaN,NaN,1,1,1
9,2011-02-07,11102,Monday,3,2,2011,10,NaN,NaN,NaN,NaN,1,1,0


It seems that for col 'wm_yr_wk', the first three digits is the year, the last two is the week.

In [22]:
df_event = df_cal.loc[:,['wm_yr_wk', 'event_type_1', 'event_type_2']]
df_event.head()

,wm_yr_wk,event_type_1,event_type_2
0,11101,NaN,NaN
1,11101,NaN,NaN
2,11101,NaN,NaN
3,11101,NaN,NaN
4,11101,NaN,NaN


In [23]:
df_event = pd.get_dummies(df_event)
df_event.head()

,wm_yr_wk,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_type_2_Cultural,event_type_2_Religious
0,11101,0,0,0,0,0,0
1,11101,0,0,0,0,0,0
2,11101,0,0,0,0,0,0
3,11101,0,0,0,0,0,0
4,11101,0,0,0,0,0,0


In [24]:
df_event = df_event.groupby(['wm_yr_wk'], as_index=False).sum()

In [25]:
df_event.head()

,wm_yr_wk,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_type_2_Cultural,event_type_2_Religious
0,11101,0,0,0,0,0,0
1,11102,0,0,0,1,0,0
2,11103,1,0,0,0,0,0
3,11104,0,1,0,0,0,0
4,11105,0,0,0,0,0,0


In [26]:
df_event.describe()

,wm_yr_wk,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_type_2_Cultural,event_type_2_Religious
count,282.000000,282.000000,282.000000,282.000000,282.000000,282.000000,282.000000
mean,11347.780142,0.131206,0.184397,0.195035,0.063830,0.014184,0.003546
std,155.927718,0.338225,0.388497,0.431306,0.244884,0.118461,0.059549
min,11101.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11219.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,11337.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,11502.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,11621.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000


In [27]:
(df_event.event_type_1_Religious == 2).sum()

4

In [28]:
df_day = df_day.merge(df_cal.loc[:,['wm_yr_wk','d']],left_on='day',right_on='d')

In [29]:
df_day.drop(['day','d'],axis=1,inplace=True)
df_day.rename(columns={'week':'week_id'},inplace=True)

In [30]:
df_day.head()

,item_id,dept_id,store_id,sales,week_id,wm_yr_wk
0,HOBBIES_1_001,HOBBIES_1,CA_1,0,1,11101
1,HOBBIES_1_002,HOBBIES_1,CA_1,0,1,11101
2,HOBBIES_1_003,HOBBIES_1,CA_1,0,1,11101
3,HOBBIES_1_004,HOBBIES_1,CA_1,0,1,11101
4,HOBBIES_1_005,HOBBIES_1,CA_1,0,1,11101


In [31]:
df_week = df_day.groupby(['item_id','wm_yr_wk','dept_id','store_id'], as_index=False).sum()

In [32]:
df_week_copy = df_week.copy()
#df_week = df_week_copy.copy()

In [33]:
df_week.drop(['week_id'],axis=1,inplace=True)

In [34]:
df_week.shape

(8476220, 5)

In [35]:
df_week.head(10)

,item_id,wm_yr_wk,dept_id,store_id,sales
0,FOODS_1_001,11101,FOODS_1,CA_1,10
1,FOODS_1_001,11101,FOODS_1,CA_2,11
2,FOODS_1_001,11101,FOODS_1,CA_3,8
3,FOODS_1_001,11101,FOODS_1,CA_4,5
4,FOODS_1_001,11101,FOODS_1,TX_1,2
5,FOODS_1_001,11101,FOODS_1,TX_2,7
6,FOODS_1_001,11101,FOODS_1,TX_3,0
7,FOODS_1_001,11101,FOODS_1,WI_1,8
8,FOODS_1_001,11101,FOODS_1,WI_2,3
9,FOODS_1_001,11101,FOODS_1,WI_3,3


In [36]:
278*30490

8476220

In [37]:
df_price = pd.read_csv('../Team-1/Retail Store Sales Forecast and Accuracy Analysis/Data/sell_prices.csv')

In [38]:
df_price.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [39]:
df_price.shape

(6841121, 4)

In [40]:
df_week = df_week.merge(df_price, how='left' ,left_on=['store_id', 'item_id', 'wm_yr_wk'], right_on=['store_id', 'item_id', 'wm_yr_wk'])

In [41]:
df_week = df_week.merge(df_event, how='left', left_on='wm_yr_wk', right_on='wm_yr_wk')

In [42]:
df_week.head(10)

,item_id,wm_yr_wk,dept_id,store_id,sales,sell_price,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_type_2_Cultural,event_type_2_Religious
0,FOODS_1_001,11101,FOODS_1,CA_1,10,2.0,0,0,0,0,0,0
1,FOODS_1_001,11101,FOODS_1,CA_2,11,2.0,0,0,0,0,0,0
2,FOODS_1_001,11101,FOODS_1,CA_3,8,2.0,0,0,0,0,0,0
3,FOODS_1_001,11101,FOODS_1,CA_4,5,2.0,0,0,0,0,0,0
4,FOODS_1_001,11101,FOODS_1,TX_1,2,2.0,0,0,0,0,0,0
5,FOODS_1_001,11101,FOODS_1,TX_2,7,2.0,0,0,0,0,0,0
6,FOODS_1_001,11101,FOODS_1,TX_3,0,NaN,0,0,0,0,0,0
7,FOODS_1_001,11101,FOODS_1,WI_1,8,2.0,0,0,0,0,0,0
8,FOODS_1_001,11101,FOODS_1,WI_2,3,2.0,0,0,0,0,0,0
9,FOODS_1_001,11101,FOODS_1,WI_3,3,2.0,0,0,0,0,0,0


In [43]:
df_week.shape

(8476220, 12)

Aggregate by store:

In [44]:
df_week['revenue'] = df_week.apply(lambda x: x.sales * x.sell_price, axis=1)

In [45]:
df_week.head(10)

,item_id,wm_yr_wk,dept_id,store_id,sales,sell_price,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_type_2_Cultural,event_type_2_Religious,revenue
0,FOODS_1_001,11101,FOODS_1,CA_1,10,2.0,0,0,0,0,0,0,20.0
1,FOODS_1_001,11101,FOODS_1,CA_2,11,2.0,0,0,0,0,0,0,22.0
2,FOODS_1_001,11101,FOODS_1,CA_3,8,2.0,0,0,0,0,0,0,16.0
3,FOODS_1_001,11101,FOODS_1,CA_4,5,2.0,0,0,0,0,0,0,10.0
4,FOODS_1_001,11101,FOODS_1,TX_1,2,2.0,0,0,0,0,0,0,4.0
5,FOODS_1_001,11101,FOODS_1,TX_2,7,2.0,0,0,0,0,0,0,14.0
6,FOODS_1_001,11101,FOODS_1,TX_3,0,NaN,0,0,0,0,0,0,NaN
7,FOODS_1_001,11101,FOODS_1,WI_1,8,2.0,0,0,0,0,0,0,16.0
8,FOODS_1_001,11101,FOODS_1,WI_2,3,2.0,0,0,0,0,0,0,6.0
9,FOODS_1_001,11101,FOODS_1,WI_3,3,2.0,0,0,0,0,0,0,6.0


In [46]:
df_week['sell_price'].isna().sum()

1757059

In [47]:
df_week['sales'].isna().sum()

0

In [48]:
((df_week['sell_price'].isna()) & (df_week['sales'] != 0)).sum()

0

In [49]:
df_week_store = df_week.drop(['item_id', 'dept_id', 'sales', 'sell_price'], axis=1)
df_week_store.head(10)

,wm_yr_wk,store_id,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_type_2_Cultural,event_type_2_Religious,revenue
0,11101,CA_1,0,0,0,0,0,0,20.0
1,11101,CA_2,0,0,0,0,0,0,22.0
2,11101,CA_3,0,0,0,0,0,0,16.0
3,11101,CA_4,0,0,0,0,0,0,10.0
4,11101,TX_1,0,0,0,0,0,0,4.0
5,11101,TX_2,0,0,0,0,0,0,14.0
6,11101,TX_3,0,0,0,0,0,0,NaN
7,11101,WI_1,0,0,0,0,0,0,16.0
8,11101,WI_2,0,0,0,0,0,0,6.0
9,11101,WI_3,0,0,0,0,0,0,6.0


In [50]:
df_week_store = df_week_store.groupby(['store_id','wm_yr_wk'], as_index=False).sum()

In [51]:
df_week_store.shape

(2780, 9)

In [52]:
df_week_store.head()

,store_id,wm_yr_wk,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_type_2_Cultural,event_type_2_Religious,revenue
0,CA_1,11101,0.0,0.0,0.0,0.0,0.0,0.0,58387.95
1,CA_1,11102,0.0,0.0,0.0,3049.0,0.0,0.0,62806.71
2,CA_1,11103,3049.0,0.0,0.0,0.0,0.0,0.0,57720.15
3,CA_1,11104,0.0,3049.0,0.0,0.0,0.0,0.0,56766.79
4,CA_1,11105,0.0,0.0,0.0,0.0,0.0,0.0,55249.84


In [53]:
df_week_store.describe()

,wm_yr_wk,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_type_2_Cultural,event_type_2_Religious,revenue
count,2780.000000,2780.000000,2780.000000,2780.000000,2780.000000,2780.000000,2780.000000,2780.000000
mean,11343.870504,405.802158,559.348921,592.251799,175.482014,32.902878,10.967626,68912.786345
std,153.318801,1035.857790,1180.290114,1312.690257,710.233887,315.077744,182.570514,24362.145284
min,11101.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21631.560000
25%,11218.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,53435.250000
50%,11335.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,64047.895000
75%,11452.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,81298.240000
max,11617.000000,3049.000000,3049.000000,6098.000000,3049.000000,3049.000000,3049.000000,153107.450000


In [54]:
df_week_store[['event_type_1_Cultural', 'event_type_1_National', 'event_type_1_Religious', 'event_type_1_Sporting', 'event_type_2_Cultural', 'event_type_2_Religious']] = \
(df_week_store[['event_type_1_Cultural', 'event_type_1_National', 'event_type_1_Religious', 'event_type_1_Sporting', 'event_type_2_Cultural', 'event_type_2_Religious']]\
 /3049).astype(int)

In [55]:
df_week_store.store_id.nunique()

10

In [56]:
df_dummy = pd.get_dummies(df_week_store['store_id'], prefix='store_id').iloc[:,:-1]

In [57]:
df_week_store = pd.concat([df_week_store.iloc[:,1:], df_dummy], axis=1)

In [58]:
df_week_store.head(10)

,wm_yr_wk,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_type_2_Cultural,event_type_2_Religious,revenue,store_id_CA_1,store_id_CA_2,store_id_CA_3,store_id_CA_4,store_id_TX_1,store_id_TX_2,store_id_TX_3,store_id_WI_1,store_id_WI_2
0,11101,0,0,0,0,0,0,58387.95,1,0,0,0,0,0,0,0,0
1,11102,0,0,0,1,0,0,62806.71,1,0,0,0,0,0,0,0,0
2,11103,1,0,0,0,0,0,57720.15,1,0,0,0,0,0,0,0,0
3,11104,0,1,0,0,0,0,56766.79,1,0,0,0,0,0,0,0,0
4,11105,0,0,0,0,0,0,55249.84,1,0,0,0,0,0,0,0,0
5,11106,0,0,1,0,0,0,61073.86,1,0,0,0,0,0,0,0,0
6,11107,1,0,1,0,0,0,56987.73,1,0,0,0,0,0,0,0,0
7,11108,0,0,1,0,0,0,54975.57,1,0,0,0,0,0,0,0,0
8,11109,0,0,0,0,0,0,56900.63,1,0,0,0,0,0,0,0,0
9,11110,0,0,0,0,0,0,64769.65,1,0,0,0,0,0,0,0,0


In [59]:
df_week_store.shape

(2780, 17)

In [60]:
df_week_store.isna().sum()

wm_yr_wk                  0
event_type_1_Cultural     0
event_type_1_National     0
event_type_1_Religious    0
event_type_1_Sporting     0
event_type_2_Cultural     0
event_type_2_Religious    0
revenue                   0
store_id_CA_1             0
store_id_CA_2             0
store_id_CA_3             0
store_id_CA_4             0
store_id_TX_1             0
store_id_TX_2             0
store_id_TX_3             0
store_id_WI_1             0
store_id_WI_2             0
dtype: int64